In [1]:
import h5py
import tensorflow as tf
import numpy as np
import math
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models, Model, Input


In [2]:
hdf5_file = "cal_data.h5"

In [3]:
def hdf5_generator():
    with h5py.File(hdf5_file, 'r') as f:
        data = f['data']
        labels = f['labels']
        num_samples = data.shape[0]
        
        for i in range(num_samples):
            yield data[i], labels[i]

In [4]:
def _fixup_shape(images, labels):
    images.set_shape([32, 20, 110, 11])
    labels.set_shape([32,])
    return images, labels

In [5]:
dataset = tf.data.Dataset.from_generator(
    hdf5_generator,
    output_signature=(
        tf.TensorSpec(shape=(20, 110, 11), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int16)
    )
)
# Optimize the pipeline
dataset = (dataset
           .batch(32,drop_remainder=True)
           .map(_fixup_shape)
           .prefetch(tf.data.AUTOTUNE))
print("Data loaded")

Data loaded


In [7]:
model = models.Sequential([
    # Input layer
    layers.Input(shape=(20, 110, 11, 1)),
    
    # First 3D Convolutional layer
    layers.Conv3D(32, kernel_size=(3, 3, 3), activation='relu'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    
    # Second 3D Convolutional layer
    layers.Conv3D(64, kernel_size=(3, 3, 3), activation='relu'),
    layers.MaxPooling3D(pool_size=(2, 2, 2)),
    
    # Flatten the 3D features
    layers.Flatten(),
    
    # Fully connected layers
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    
    # Output layer (e.g., regression output or classification)
    layers.Dense(1, activation='linear')  # 'linear' for regression tasks
])

# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # or 'categorical_crossentropy' for classification
              metrics=['mean_absolute_error'])

In [8]:
print("Training")
#Trening
history = model.fit(
    dataset,
    epochs=12,      # Number of training epochs
    #batch_size=16,  # Number of samples per batch - !! maybe too much because data is already batched !!
    shuffle=True,   # Shuffle training data at each epoch
    verbose=1       # Show training progress (1 = progress bar, 2 = epoch-only)
)
print("Training done")

Training
Epoch 1/12
  14062/Unknown 1499s 105ms/step - loss: 98.2634 - mean_absolute_error: 4.5018

InvalidArgumentError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "C:\Users\przem\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Users\przem\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "c:\Users\przem\OneDrive\Desktop\VS_explorer\projekt_studencki\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\przem\OneDrive\Desktop\VS_explorer\projekt_studencki\.venv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\przem\OneDrive\Desktop\VS_explorer\projekt_studencki\.venv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\przem\OneDrive\Desktop\VS_explorer\projekt_studencki\.venv\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\przem\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py"